In [1]:
import json 
import tensorflow as tf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras import backend as K
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPooling1D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import SpatialDropout1D, concatenate, BatchNormalization
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.utils import plot_model

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Importer les Train et test pour les abstract 

In [2]:
data_dir = "data/abstract_only"
feature = "abstract"
label = 'label'
x_train = pd.read_csv(f"{data_dir}/train.csv")[feature]
y_train = pd.read_csv(f"{data_dir}/train.csv")[label]

x_test = pd.read_csv(f"{data_dir}/test.csv")[feature]
y_test = pd.read_csv(f"{data_dir}/test.csv")[label]

x_train.shape, y_train.shape, x_test.shape, y_test.shape,

((8000,), (8000,), (2000,), (2000,))

In [3]:
#manipulation de y:catégorie: label
def create_vocab(dt): 
  to_id = {'<PAD>': 0, '<UNK>':1}

  for sent in dt:
    for w in sent: 
      if w not in to_id.keys():
        to_id[w] = len(to_id)

  from_id = {v: k for k, v in to_id.items()}

  vocab = len(to_id.keys())

  return to_id, from_id, vocab

def preprocess_Y(Y, cat_to_id): 
  res = []
  for ex in Y: 
    if ex not in cat_to_id.keys():
      res.append(cat_to_id['<UNK>'])
    else:
      res.append(cat_to_id[ex])
  return np.array(res)

cat_to_id, cat_from_id, cat_vocab = create_vocab([y_train])
y_train_id = preprocess_Y(y_train, cat_to_id)
y_test_id = preprocess_Y(y_test, cat_to_id)

#Manipulation des titres 
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq =  tokenizer.texts_to_sequences(x_test)

x_train_seq = tf.keras.preprocessing.sequence.pad_sequences(x_train_seq, maxlen = 888, truncating='post')
x_test_seq = tf.keras.preprocessing.sequence.pad_sequences(x_test_seq, maxlen = 888, truncating='post')

In [4]:
#une fonction pour visualiser accuracy et loss
def plot_history(history, metric="acc"):
    # summarize history for accuracy
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.title(f'model {metric}')
    plt.ylabel(metric)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [5]:
# Fonction permettant de charger un embedding 

import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def load_glove_embeddings(fp, embedding_dim, include_empty_char=True):
    """
    Loads pre-trained word embeddings (GloVe embeddings)
        Inputs: - fp: filepath of pre-trained glove embeddings
                - embedding_dim: dimension of each vector embedding
                - generate_matrix: whether to generate an embedding matrix
        Outputs:
                - word2coefs: Dictionary. Word to its corresponding coefficients
                - word2index: Dictionary. Word to word-index
                - embedding_matrix: Embedding matrix for Keras Embedding layer
    """
    # First, build the "word2coefs" and "word2index"
    word2coefs = {} # word to its corresponding coefficients
    word2index = {} # word to word-index
    with open(fp) as f:
        for idx, line in enumerate(f):
            try:
                data = [x.strip().lower() for x in line.split()]
                word = data[0]
                coefs = np.asarray(data[1:embedding_dim+1], dtype='float32')
                word2coefs[word] = coefs
                if word not in word2index:
                    word2index[word] = len(word2index)
            except Exception as e:
                print('Exception occurred in `load_glove_embeddings`:', e)
                continue
        # End of for loop.
    # End of with open
    if include_empty_char:
        word2index[''] = len(word2index)
    # Second, build the "embedding_matrix"
    # Words not found in embedding index will be all-zeros. Hence, the "+1".
    vocab_size = len(word2coefs)+1 if include_empty_char else len(word2coefs)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word2index.items():
        embedding_vec = word2coefs.get(word)
        if embedding_vec is not None and embedding_vec.shape[0]==embedding_dim:
            embedding_matrix[idx] = np.asarray(embedding_vec)
    # return word2coefs, word2index, embedding_matrix
    return word2index, np.asarray(embedding_matrix)

[nltk_data] Downloading package punkt to /home/ahlem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Charger les embeddings à l'aide de la fonction load_glove_embeddings

word2index, embedding_matrix = load_glove_embeddings('glove.6B.50d.txt', embedding_dim=50)

In [7]:
# ecrire une fonction de tokenization custom pour preprocesser les textes

def custom_tokenize(doc):
  res = []
  for ex in word_tokenize(doc): 
    if ex not in word2index.keys():
      res.append(word2index['unk'])
    else:
      res.append(word2index[ex])
  return np.array(res)
    
# Encoder les textes avec la fonction custom
X_train_glove = [custom_tokenize(x) for x in x_train]
print(X_train_glove[0])

X_test_glove = [custom_tokenize(x) for x in x_test]
print(X_test_glove[0])


[  9247   4415   7693  11231     23   7215     24      6      0   9247
   6723     14      7   1013   2741    747    445      4   5998  26431
     46  10645  12168      3    933      5  23925  11231   4276     10
     37  11231    747     17    333   2741   5306      2   2317      1
    197      4   6326      0   9247   4415   7693    747     17    333
     23  10645     24     46  21699    933   8931     56    999      6
      0   9247    610      2     10   3568      0  11231   9009      3
   9247   4415   7693      1     37   1307  10388      7   1999 226671
   2741  12597    622      0  12886  24113  26479   1587      4  33062
      0  12838      3  21566   4341      3  19847   2817      1     42
     14    175      0 201534   2741  12597      2      0  14575    604
     17      0   9247   4415   7693  75816    273     12      1     21
  54193      0  21566  12838      1      0   1225  12597     31    219
  35743    883      5 112878 226671      5      0     68   6223      6
   107

In [8]:
# Padding des sequences

X_train_glove = tf.keras.preprocessing.sequence.pad_sequences(X_train_glove, maxlen = 888, truncating='post')
X_test_glove = tf.keras.preprocessing.sequence.pad_sequences(X_test_glove, maxlen = 888, truncating='post')

# CNN
## Modèle basique
## Embedding non préentrainés

In [9]:
embed_dim = 128
batch_size = 128

model = tf.keras.models.Sequential() # modèle séquentiel
model.add(tf.keras.layers.Embedding(10000, embed_dim,input_length = 888))# couche d'embedding de taille 128
model.add(tf.keras.layers.Conv1D(32,
                 2,
                 padding='valid',
                 activation='relu',
                 strides=1))

model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dense(cat_vocab))
model.add(tf.keras.layers.Activation('softmax'))

# Compiler le modèle 

model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

# Afficher le summary du modèle
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 888, 128)          1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 887, 32)           8224      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 93)                11997     
__________

In [11]:
# Fitter le modèle 

history = model.fit(x_train_seq, y_train_id, batch_size = batch_size, epochs = 10, validation_data=(x_test_seq, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 277us/step - loss: 3.8483 - acc: 0.1804 - val_loss: 3.3909 - val_acc: 0.1975
Epoch 2/10
8000/8000 [==============================] - 0s 53us/step - loss: 3.2230 - acc: 0.2460 - val_loss: 3.0656 - val_acc: 0.2740
Epoch 3/10
8000/8000 [==============================] - 0s 58us/step - loss: 2.9124 - acc: 0.3006 - val_loss: 2.8065 - val_acc: 0.3280
Epoch 4/10
8000/8000 [==============================] - 0s 58us/step - loss: 2.6171 - acc: 0.3593 - val_loss: 2.6141 - val_acc: 0.3600
Epoch 5/10
8000/8000 [==============================] - 0s 58us/step - loss: 2.3509 - acc: 0.4086 - val_loss: 2.5057 - val_acc: 0.3735
Epoch 6/10
8000/8000 [==============================] - 0s 55us/step - loss: 2.1099 - acc: 0.4556 - val_loss: 2.4455 - val_acc: 0.3950
Epoch 7/10
8000/8000 [==============================] - 0s 52us/step - loss: 1.8561 - acc: 0.5154 - val_loss: 2.4428 - val_acc: 0.3870
Epoch 

In [ ]:
#visualiser
plot_history(history)

In [12]:
# Evaluer le modèle
model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 0s 34us/step


[2.65889536857605, 0.38]

# CNN
## Modèle basique
## Embedding GloVE

In [13]:
# Entrainer un modèle en chargeant les poids des embeddings dans le layer Embedding

#embed_dim = 50

batch_size = 128

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 888,
                                    weights=[embedding_matrix], 
                              trainable=True))
model.add(tf.keras.layers.Conv1D(32,
                 2,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(tf.keras.layers.GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dense(cat_vocab))
model.add(tf.keras.layers.Activation('softmax'))

adam = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=adam, metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 888, 50)           20000050  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 887, 32)           3232      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               4224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 93)                11997     
__________

In [16]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 15, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 1s 96us/step - loss: 1.5855 - acc: 0.5769 - val_loss: 2.4230 - val_acc: 0.3880
Epoch 2/5
8000/8000 [==============================] - 1s 95us/step - loss: 1.4830 - acc: 0.6019 - val_loss: 2.4602 - val_acc: 0.3890
Epoch 3/5
8000/8000 [==============================] - 1s 95us/step - loss: 1.3718 - acc: 0.6302 - val_loss: 2.5146 - val_acc: 0.3835
Epoch 4/5
8000/8000 [==============================] - 1s 94us/step - loss: 1.2926 - acc: 0.6521 - val_loss: 2.5514 - val_acc: 0.3785
Epoch 5/5
8000/8000 [==============================] - 1s 95us/step - loss: 1.1854 - acc: 0.6760 - val_loss: 2.5885 - val_acc: 0.3740


In [ ]:
# Evaluer le modèle
model.evaluate(X_test_glove, y_test_id)

# CNN
## Modèle basique, Changement hyperparamètres
## Embedding non préentrainés

In [17]:
embed_dim = 128
batch_size = 128

model = tf.keras.models.Sequential() # modèle séquentiel
model.add(tf.keras.layers.Embedding(10000, embed_dim,input_length = 888))# couche d'embedding de taille 
model.add(tf.keras.layers.Conv1D(256,
                 2,
                 padding='valid',
                 activation='relu',
                 strides=1))

model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(cat_vocab))
model.add(tf.keras.layers.Activation('softmax'))

# Compiler le modèle 
adam = tf.keras.optimizers.Adam(lr=1e-4)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=adam ,metrics = ['accuracy'])
# Fitter le modèle 
history = model.fit(x_train_seq, y_train_id, batch_size = batch_size, epochs = 50, validation_data=(x_test_seq, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 2s 210us/step - loss: 4.4406 - acc: 0.1318 - val_loss: 4.3187 - val_acc: 0.1880
Epoch 2/50
8000/8000 [==============================] - 1s 159us/step - loss: 4.1282 - acc: 0.1879 - val_loss: 3.8655 - val_acc: 0.1880
Epoch 3/50
8000/8000 [==============================] - 1s 158us/step - loss: 3.7127 - acc: 0.1874 - val_loss: 3.5469 - val_acc: 0.1880
Epoch 4/50
8000/8000 [==============================] - 1s 160us/step - loss: 3.5647 - acc: 0.1846 - val_loss: 3.4735 - val_acc: 0.1880
Epoch 5/50
8000/8000 [==============================] - 1s 158us/step - loss: 3.5142 - acc: 0.1908 - val_loss: 3.4342 - val_acc: 0.1880
Epoch 6/50
8000/8000 [==============================] - 1s 160us/step - loss: 3.4634 - acc: 0.2014 - val_loss: 3.3882 - val_acc: 0.2095
Epoch 7/50
8000/8000 [==============================] - 1s 159us/step - loss: 3.4122 - acc: 0.2223 - val_loss: 3.3301 - val_acc: 0.2375


In [18]:
# Evaluer le modèle
model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 0s 98us/step


[2.2557374572753908, 0.437]

# CNN
## Modèle basique, Changement hyperparamètres
## Embedding GloVe

In [20]:
#embed_dim = 50
batch_size = 128

model = tf.keras.models.Sequential() # modèle séquentiel
model.add(tf.keras.layers.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 888,
                                    weights=[embedding_matrix], 
                              trainable=True))# couche d'embedding de taille 
model.add(tf.keras.layers.Conv1D(256,
                 2,
                 padding='valid',
                 activation='relu',
                 strides=1))


model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(cat_vocab))
model.add(tf.keras.layers.Activation('softmax'))

# Compiler le modèle 
adam = tf.keras.optimizers.Adam(lr=1e-4)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=adam ,metrics = ['accuracy'])
# Fitter le modèle 
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 100, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/70
8000/8000 [==============================] - 2s 279us/step - loss: 4.3296 - acc: 0.0999 - val_loss: 3.8339 - val_acc: 0.1880
Epoch 2/70
8000/8000 [==============================] - 1s 171us/step - loss: 3.8327 - acc: 0.1678 - val_loss: 3.5948 - val_acc: 0.1880
Epoch 3/70
8000/8000 [==============================] - 1s 176us/step - loss: 3.6888 - acc: 0.1771 - val_loss: 3.4987 - val_acc: 0.1880
Epoch 4/70
8000/8000 [==============================] - 1s 172us/step - loss: 3.6042 - acc: 0.1884 - val_loss: 3.4376 - val_acc: 0.1905
Epoch 5/70
8000/8000 [==============================] - 1s 171us/step - loss: 3.5318 - acc: 0.1964 - val_loss: 3.3829 - val_acc: 0.2030
Epoch 6/70
8000/8000 [==============================] - 1s 171us/step - loss: 3.4791 - acc: 0.2080 - val_loss: 3.3332 - val_acc: 0.2475
Epoch 7/70
8000/8000 [==============================] - 1s 175us/step - loss: 3.4117 - acc: 0.2243 - val_loss: 3.2774 - val_acc: 0.2620


8000/8000 [==============================] - 1s 176us/step - loss: 1.9290 - acc: 0.4918 - val_loss: 2.1574 - val_acc: 0.4310
Epoch 61/70
8000/8000 [==============================] - 1s 176us/step - loss: 1.9127 - acc: 0.4975 - val_loss: 2.1518 - val_acc: 0.4305
Epoch 62/70
8000/8000 [==============================] - 1s 172us/step - loss: 1.8917 - acc: 0.5030 - val_loss: 2.1456 - val_acc: 0.4300
Epoch 63/70
8000/8000 [==============================] - 1s 172us/step - loss: 1.8700 - acc: 0.5039 - val_loss: 2.1411 - val_acc: 0.4280
Epoch 64/70
8000/8000 [==============================] - 1s 174us/step - loss: 1.8675 - acc: 0.5046 - val_loss: 2.1365 - val_acc: 0.4365
Epoch 65/70
8000/8000 [==============================] - 1s 174us/step - loss: 1.8387 - acc: 0.5166 - val_loss: 2.1299 - val_acc: 0.4330
Epoch 66/70
8000/8000 [==============================] - 1s 174us/step - loss: 1.8332 - acc: 0.5119 - val_loss: 2.1229 - val_acc: 0.4375
Epoch 67/70
8000/8000 [==============================

In [26]:
# Evaluer le modèle
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 0s 138us/step


[2.0607304592132567, 0.448]

# CNN
## Modèle CNN multi-channel
## Embedding non préentrainés

In [28]:
def get_cnn_model():
    embedding_dim = 300
    filter_sizes = [2, 3, 5]
    num_filters = 256
    drop = 0.3
    MAX_LENGTH=888
    MAX_NB_WORDS = 10000
    
    inputs = Input(shape=(MAX_LENGTH,), dtype='int32')
    embedding = Embedding(input_dim=MAX_NB_WORDS,
                                output_dim=embedding_dim,
                                input_length=MAX_LENGTH)(inputs)

    reshape = Reshape((MAX_LENGTH, embedding_dim, 1))(embedding)
    conv_0 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[0], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)

    conv_1 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[1], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)
    conv_2 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[2], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)

    maxpool_0 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[0] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_0)

    maxpool_1 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[1] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_1)

    maxpool_2 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[2] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_2)
    concatenated_tensor = Concatenate(axis=1)(
        [maxpool_0, maxpool_1, maxpool_2])
    flatten = Flatten()(concatenated_tensor)
    
    dropout = Dropout(drop)(flatten)
    
    #outpout1=Dense(units=256, activation='relu')(dropout)
    #dropout1 = Dropout(drop)(outpout1)
    
    output = Dense(units=cat_vocab, activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=output)
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

cnn_model_multi_channel = get_cnn_model()

batch_size = 128
epochs = 100
history = cnn_model_multi_channel.fit(x=x_train_seq, 
                    y=y_train_id, 
                    batch_size=batch_size, 
                    validation_data=(x_test_seq, y_test_id),
                    epochs=epochs)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 11s 1ms/step - loss: 4.2572 - acc: 0.0695 - val_loss: 3.8934 - val_acc: 0.1950
Epoch 2/100
8000/8000 [==============================] - 11s 1ms/step - loss: 3.6274 - acc: 0.1891 - val_loss: 3.4757 - val_acc: 0.1880
Epoch 3/100
8000/8000 [==============================] - 11s 1ms/step - loss: 3.4259 - acc: 0.1899 - val_loss: 3.4224 - val_acc: 0.1880
Epoch 4/100
8000/8000 [==============================] - 11s 1ms/step - loss: 3.3694 - acc: 0.2009 - val_loss: 3.3794 - val_acc: 0.2015
Epoch 5/100
8000/8000 [==============================] - 11s 1ms/step - loss: 3.3017 - acc: 0.2284 - val_loss: 3.3243 - val_acc: 0.2295
Epoch 6/100
8000/8000 [==============================] - 11s 1ms/step - loss: 3.2279 - acc: 0.2487 - val_loss: 3.2544 - val_acc: 0.2480
Epoch 7/100
8000/8000 [==============================] - 11s 1ms/step - loss: 3.1429 - acc: 0.2672 - val_loss: 3.1831 - val_acc: 0.2670


8000/8000 [==============================] - 11s 1ms/step - loss: 0.7995 - acc: 0.8296 - val_loss: 2.0135 - val_acc: 0.4775
Epoch 61/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.7697 - acc: 0.8464 - val_loss: 2.0084 - val_acc: 0.4840
Epoch 62/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.7491 - acc: 0.8484 - val_loss: 2.0037 - val_acc: 0.4800
Epoch 63/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.7166 - acc: 0.8554 - val_loss: 1.9991 - val_acc: 0.4770
Epoch 64/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.6915 - acc: 0.8629 - val_loss: 1.9964 - val_acc: 0.4810
Epoch 65/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.6695 - acc: 0.8709 - val_loss: 1.9916 - val_acc: 0.4840
Epoch 66/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.6347 - acc: 0.8798 - val_loss: 1.9901 - val_acc: 0.4830
Epoch 67/100
8000/8000 [==============================

In [29]:
# Evaluer le modèle
cnn_model_multi_channel.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 1s 469us/step


[1.9862929067611694, 0.502]

In [ ]:
plot_history(history)

# CNN
## Modèle CNN multi-channel
## Embedding GloVe

In [31]:
def get_cnn_model():
    embedding_dim = 50
    filter_sizes = [2, 3, 5]
    num_filters = 256
    drop = 0.3
    MAX_LENGTH=888
    MAX_NB_WORDS = 10000
    
    inputs = Input(shape=(MAX_LENGTH,), dtype='int32')
    embedding = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 888,
                                    weights=[embedding_matrix], 
                              trainable=True)(inputs)

    reshape = Reshape((MAX_LENGTH, embedding_dim, 1))(embedding)
    conv_0 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[0], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)

    conv_1 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[1], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)
    conv_2 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[2], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)

    maxpool_0 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[0] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_0)

    maxpool_1 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[1] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_1)

    maxpool_2 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[2] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_2)
    concatenated_tensor = Concatenate(axis=1)(
        [maxpool_0, maxpool_1, maxpool_2])
    flatten = Flatten()(concatenated_tensor)
    
    dropout = Dropout(drop)(flatten)
    
    #outpout1=Dense(units=256, activation='relu')(dropout)
    #dropout1 = Dropout(drop)(outpout1)
    
    output = Dense(units=cat_vocab, activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=output)
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = get_cnn_model()
batch_size = 128

In [32]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 100, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 8s 944us/step - loss: 4.2019 - acc: 0.1118 - val_loss: 3.4187 - val_acc: 0.1935
Epoch 2/100
8000/8000 [==============================] - 6s 787us/step - loss: 3.6717 - acc: 0.1575 - val_loss: 3.3163 - val_acc: 0.2275
Epoch 3/100
8000/8000 [==============================] - 6s 788us/step - loss: 3.5461 - acc: 0.1861 - val_loss: 3.2393 - val_acc: 0.2460
Epoch 4/100
8000/8000 [==============================] - 6s 788us/step - loss: 3.4448 - acc: 0.2075 - val_loss: 3.1699 - val_acc: 0.2750
Epoch 5/100
8000/8000 [==============================] - 6s 788us/step - loss: 3.3410 - acc: 0.2311 - val_loss: 3.1051 - val_acc: 0.2820
Epoch 6/100
8000/8000 [==============================] - 6s 785us/step - loss: 3.2622 - acc: 0.2454 - val_loss: 3.0483 - val_acc: 0.2875
Epoch 7/100
8000/8000 [==============================] - 6s 787us/step - loss: 3.1695 - acc: 0.2635 - val_loss: 2.9905 - val_acc: 

8000/8000 [==============================] - 6s 785us/step - loss: 1.4669 - acc: 0.6034 - val_loss: 2.0548 - val_acc: 0.4370
Epoch 61/100
8000/8000 [==============================] - 6s 788us/step - loss: 1.4511 - acc: 0.6086 - val_loss: 2.0509 - val_acc: 0.4370
Epoch 62/100
8000/8000 [==============================] - 6s 789us/step - loss: 1.4324 - acc: 0.6169 - val_loss: 2.0433 - val_acc: 0.4405
Epoch 63/100
8000/8000 [==============================] - 6s 790us/step - loss: 1.4143 - acc: 0.6219 - val_loss: 2.0406 - val_acc: 0.4420
Epoch 64/100
8000/8000 [==============================] - 6s 790us/step - loss: 1.3803 - acc: 0.6286 - val_loss: 2.0342 - val_acc: 0.4450
Epoch 65/100
8000/8000 [==============================] - 6s 788us/step - loss: 1.3791 - acc: 0.6296 - val_loss: 2.0280 - val_acc: 0.4410
Epoch 66/100
8000/8000 [==============================] - 6s 787us/step - loss: 1.3559 - acc: 0.6411 - val_loss: 2.0236 - val_acc: 0.4465
Epoch 67/100
8000/8000 [=======================

In [33]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 20, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/20
8000/8000 [==============================] - 6s 789us/step - loss: 0.8192 - acc: 0.8050 - val_loss: 1.9205 - val_acc: 0.4715
Epoch 2/20
8000/8000 [==============================] - 6s 785us/step - loss: 0.8078 - acc: 0.8035 - val_loss: 1.9203 - val_acc: 0.4730
Epoch 3/20
8000/8000 [==============================] - 6s 785us/step - loss: 0.7850 - acc: 0.8126 - val_loss: 1.9235 - val_acc: 0.4660
Epoch 4/20
8000/8000 [==============================] - 6s 787us/step - loss: 0.7799 - acc: 0.8110 - val_loss: 1.9189 - val_acc: 0.4770
Epoch 5/20
8000/8000 [==============================] - 6s 787us/step - loss: 0.7606 - acc: 0.8199 - val_loss: 1.9157 - val_acc: 0.4735
Epoch 6/20
8000/8000 [==============================] - 6s 788us/step - loss: 0.7457 - acc: 0.8257 - val_loss: 1.9153 - val_acc: 0.4710
Epoch 7/20
8000/8000 [==============================] - 6s 787us/step - loss: 0.7504 - acc: 0.8191 - val_loss: 1.9144 - val_acc: 0.4745


In [34]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 100, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 6s 793us/step - loss: 0.5809 - acc: 0.8705 - val_loss: 1.9087 - val_acc: 0.4760
Epoch 2/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.5698 - acc: 0.8739 - val_loss: 1.9093 - val_acc: 0.4745
Epoch 3/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.5576 - acc: 0.8795 - val_loss: 1.9091 - val_acc: 0.4765
Epoch 4/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.5458 - acc: 0.8861 - val_loss: 1.9088 - val_acc: 0.4725
Epoch 5/100
8000/8000 [==============================] - 6s 787us/step - loss: 0.5383 - acc: 0.8872 - val_loss: 1.9088 - val_acc: 0.4810
Epoch 6/100
8000/8000 [==============================] - 6s 789us/step - loss: 0.5305 - acc: 0.8891 - val_loss: 1.9104 - val_acc: 0.4730
Epoch 7/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.5106 - acc: 0.8971 - val_loss: 1.9072 - val_acc: 

8000/8000 [==============================] - 6s 787us/step - loss: 0.1823 - acc: 0.9780 - val_loss: 1.9717 - val_acc: 0.4765
Epoch 61/100
8000/8000 [==============================] - 6s 787us/step - loss: 0.1753 - acc: 0.9796 - val_loss: 1.9766 - val_acc: 0.4745
Epoch 62/100
8000/8000 [==============================] - 6s 787us/step - loss: 0.1773 - acc: 0.9776 - val_loss: 1.9769 - val_acc: 0.4770
Epoch 63/100
8000/8000 [==============================] - 6s 787us/step - loss: 0.1740 - acc: 0.9781 - val_loss: 1.9762 - val_acc: 0.4750
Epoch 64/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.1694 - acc: 0.9786 - val_loss: 1.9807 - val_acc: 0.4730
Epoch 65/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.1656 - acc: 0.9784 - val_loss: 1.9846 - val_acc: 0.4785
Epoch 66/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.1544 - acc: 0.9841 - val_loss: 1.9841 - val_acc: 0.4765
Epoch 67/100
8000/8000 [=======================

KeyboardInterrupt: 

In [ ]:
# Evaluer le modèle
model.evaluate(X_test_glove, y_test_id)

# RNN
## Modèle basique: LSTM
## Embedding non préentrainés

In [40]:
# Créer un réseau à base de LSTM avec au minimum:
# Embedding
# Dropout
# LSTM
# Dropout
# Classifieur

embed_dim = 128
lstm_out = 128
batch_size = 128

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(10000, embed_dim,input_length = 888))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_out)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(cat_vocab,activation='softmax'))

# Compiler le modèle 
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

# Afficher le summary du modèle
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 888, 128)          1280000   
_________________________________________________________________
dropout_14 (Dropout)         (None, 888, 128)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               263168    
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 93)                11997     
Total para

In [41]:
# Fitter le modèle 

history = model.fit(x_train_seq, y_train_id, batch_size = batch_size, epochs = 10, validation_data=(x_test_seq, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 107s 13ms/step - loss: 3.7580 - acc: 0.1775 - val_loss: 3.4724 - val_acc: 0.1880
Epoch 2/10
8000/8000 [==============================] - 105s 13ms/step - loss: 3.3547 - acc: 0.2256 - val_loss: 3.1398 - val_acc: 0.2505
Epoch 3/10
8000/8000 [==============================] - 105s 13ms/step - loss: 3.0287 - acc: 0.2587 - val_loss: 3.0237 - val_acc: 0.2555
Epoch 4/10
8000/8000 [==============================] - 105s 13ms/step - loss: 2.8280 - acc: 0.2886 - val_loss: 3.0150 - val_acc: 0.2735
Epoch 5/10
8000/8000 [==============================] - 105s 13ms/step - loss: 2.6424 - acc: 0.3314 - val_loss: 3.0171 - val_acc: 0.2735
Epoch 6/10
8000/8000 [==============================] - 105s 13ms/step - loss: 2.4088 - acc: 0.3761 - val_loss: 3.0495 - val_acc: 0.2705
Epoch 7/10
8000/8000 [==============================] - 105s 13ms/step - loss: 2.1742 - acc: 0.4375 - val_loss: 3.1854 - val_acc: 

In [42]:
# Evaluer le modèle
model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 31s 16ms/step


[3.5078332405090333, 0.2715]

# RNN
## Modèle basique: LSTM
## Embedding Glove

In [43]:

lstm_out = 128
batch_size = 128

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 888,
                                    weights=[embedding_matrix], 
                              trainable=True))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_out)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(cat_vocab,activation='softmax'))

# Compiler le modèle 
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

# Afficher le summary du modèle
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 888, 50)           20000050  
_________________________________________________________________
dropout_17 (Dropout)         (None, 888, 50)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               183296    
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_19 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 93)                11997     
Total para

In [44]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 10, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 110s 14ms/step - loss: 3.7265 - acc: 0.1685 - val_loss: 3.4480 - val_acc: 0.1880
Epoch 2/10
8000/8000 [==============================] - 108s 13ms/step - loss: 3.3810 - acc: 0.2121 - val_loss: 3.1523 - val_acc: 0.2470
Epoch 3/10
8000/8000 [==============================] - 108s 13ms/step - loss: 3.1553 - acc: 0.2362 - val_loss: 3.0177 - val_acc: 0.2705
Epoch 4/10
8000/8000 [==============================] - 108s 13ms/step - loss: 3.1533 - acc: 0.2477 - val_loss: 3.1173 - val_acc: 0.2380
Epoch 5/10
8000/8000 [==============================] - 108s 13ms/step - loss: 3.1419 - acc: 0.2282 - val_loss: 3.0706 - val_acc: 0.2560
Epoch 6/10
8000/8000 [==============================] - 108s 13ms/step - loss: 2.9738 - acc: 0.2570 - val_loss: 2.9205 - val_acc: 0.2645
Epoch 7/10
8000/8000 [==============================] - 108s 13ms/step - loss: 2.9735 - acc: 0.2556 - val_loss: 3.0213 - val_acc: 

In [45]:
# Evaluer le modèle
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 31s 15ms/step


[2.7848419647216796, 0.2975]

# RNN
## Modèle basique: GRU
## Embedding non préentrainés

In [55]:
def get_simple_rnn_model():
    embedding_dim = 300
    MAX_NB_WORDS = 10000
    MAX_LENGTH=888
    embedding_matrix = np.random.random((MAX_NB_WORDS, embedding_dim))
    
    inp = Input(shape=(MAX_LENGTH, ))
    x = Embedding(input_dim=MAX_NB_WORDS, output_dim=embedding_dim, input_length=MAX_LENGTH, 
                  weights=[embedding_matrix], trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(cat_vocab, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    adam = Adam(lr=1e-3)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer= adam,
                  metrics=['accuracy'])
    return model

rnn_simple_model = get_simple_rnn_model()

In [56]:
batch_size = 128
epochs = 50
history = rnn_simple_model .fit(x=x_train_seq, 
                    y=y_train_id, 
                    batch_size=batch_size, 
                    validation_data=(x_test_seq, y_test_id),
                    epochs=epochs)

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 77s 10ms/step - loss: 3.5834 - acc: 0.1781 - val_loss: 3.4761 - val_acc: 0.1880
Epoch 2/50
8000/8000 [==============================] - 74s 9ms/step - loss: 3.4467 - acc: 0.2003 - val_loss: 3.3628 - val_acc: 0.2370
Epoch 3/50
8000/8000 [==============================] - 74s 9ms/step - loss: 3.2749 - acc: 0.2525 - val_loss: 3.1657 - val_acc: 0.2660
Epoch 4/50
8000/8000 [==============================] - 74s 9ms/step - loss: 3.0723 - acc: 0.2812 - val_loss: 2.9977 - val_acc: 0.2810
Epoch 5/50
8000/8000 [==============================] - 74s 9ms/step - loss: 2.8794 - acc: 0.3119 - val_loss: 2.8271 - val_acc: 0.3265
Epoch 6/50
8000/8000 [==============================] - 74s 9ms/step - loss: 2.6928 - acc: 0.3405 - val_loss: 2.6730 - val_acc: 0.3445
Epoch 7/50
8000/8000 [==============================] - 74s 9ms/step - loss: 2.5142 - acc: 0.3776 - val_loss: 2.5746 - val_acc: 0.3660
Epoch 

In [57]:
rnn_simple_model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 22s 11ms/step


[3.0843389625549316, 0.405]

# RNN
## Modèle basique: GRU
## Embedding Glove

In [47]:
def get_simple_rnn_model():
    embedding_dim = 50
    MAX_NB_WORDS = 10000
    MAX_LENGTH=888
    
    inp = Input(shape=(MAX_LENGTH, ))
    x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 888,
                                    weights=[embedding_matrix], 
                              trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(cat_vocab, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    adam = Adam(lr=1e-3)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer= adam,
                  metrics=['accuracy'])
    return model

model = get_simple_rnn_model()

In [48]:
history = model.fit(X_train_glove, y_train_id, batch_size = 128, epochs = 50, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 75s 9ms/step - loss: 3.5725 - acc: 0.1764 - val_loss: 3.3635 - val_acc: 0.1890
Epoch 2/50
8000/8000 [==============================] - 73s 9ms/step - loss: 3.2621 - acc: 0.2371 - val_loss: 3.0685 - val_acc: 0.2480
Epoch 3/50
8000/8000 [==============================] - 73s 9ms/step - loss: 2.9927 - acc: 0.2769 - val_loss: 2.8307 - val_acc: 0.2995
Epoch 4/50
8000/8000 [==============================] - 73s 9ms/step - loss: 2.7826 - acc: 0.3120 - val_loss: 2.6298 - val_acc: 0.3295
Epoch 5/50
8000/8000 [==============================] - 73s 9ms/step - loss: 2.6059 - acc: 0.3376 - val_loss: 2.5064 - val_acc: 0.3595
Epoch 6/50
8000/8000 [==============================] - 72s 9ms/step - loss: 2.4680 - acc: 0.3634 - val_loss: 2.3704 - val_acc: 0.3830
Epoch 7/50
8000/8000 [==============================] - 72s 9ms/step - loss: 2.3506 - acc: 0.3876 - val_loss: 2.2958 - val_acc: 0.3960
Epoch 8

KeyboardInterrupt: 

In [49]:
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 20s 10ms/step


[2.0637245044708252, 0.461]

# CNN_RNN

## Embedding non préentrainés

In [51]:
def get_rnn_cnn_model():
    embedding_dim = 300
    MAX_NB_WORDS = 10000
    MAX_LENGTH=888
    embedding_matrix = np.random.random((MAX_NB_WORDS, embedding_dim))
    inp = Input(shape=(888, ))
    x = Embedding(MAX_NB_WORDS, embedding_dim, weights=[embedding_matrix], input_length=MAX_LENGTH, trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(cat_vocab, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

model = get_rnn_cnn_model()
batch_size = 128
epochs = 15
history = model.fit(x=x_train_seq, 
                    y=y_train_id, 
                    batch_size=batch_size, 
                    validation_data=(x_test_seq, y_test_id),
                    epochs=epochs)

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 77s 10ms/step - loss: 3.6610 - acc: 0.1613 - val_loss: 3.5200 - val_acc: 0.1885
Epoch 2/15
8000/8000 [==============================] - 74s 9ms/step - loss: 3.4997 - acc: 0.1903 - val_loss: 3.4395 - val_acc: 0.2140
Epoch 3/15
8000/8000 [==============================] - 74s 9ms/step - loss: 3.3313 - acc: 0.2414 - val_loss: 3.1894 - val_acc: 0.2660
Epoch 4/15
8000/8000 [==============================] - 74s 9ms/step - loss: 3.1172 - acc: 0.2742 - val_loss: 3.0598 - val_acc: 0.2795
Epoch 5/15
8000/8000 [==============================] - 74s 9ms/step - loss: 2.9716 - acc: 0.2928 - val_loss: 2.9409 - val_acc: 0.2995
Epoch 6/15
8000/8000 [==============================] - 74s 9ms/step - loss: 2.8312 - acc: 0.3170 - val_loss: 2.8344 - val_acc: 0.3130
Epoch 7/15
8000/8000 [==============================] - 74s 9ms/step - loss: 2.6704 - acc: 0.3444 - val_loss: 2.7397 - val_acc: 0.3385
Epoch 

In [52]:
model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 22s 11ms/step


[2.476172409057617, 0.3985]

# CNN_RNN

## Embedding GloVe

In [53]:
def get_rnn_cnn_model():
    embedding_dim = 50
    MAX_NB_WORDS = 10000
    MAX_LENGTH=888
    
    inp = Input(shape=(888, ))
    x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 888,
                                    weights=[embedding_matrix], 
                              trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(cat_vocab, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

model = get_rnn_cnn_model()
batch_size = 128
epochs = 15
history = model.fit(X_train_glove,
                    y_train_id, 
                    batch_size = batch_size,
                    epochs = epochs,
                    validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 76s 10ms/step - loss: 3.5766 - acc: 0.1928 - val_loss: 3.3159 - val_acc: 0.2370
Epoch 2/15
8000/8000 [==============================] - 73s 9ms/step - loss: 3.1665 - acc: 0.2623 - val_loss: 2.9289 - val_acc: 0.3060
Epoch 3/15
8000/8000 [==============================] - 73s 9ms/step - loss: 2.8595 - acc: 0.3056 - val_loss: 2.6522 - val_acc: 0.3335
Epoch 4/15
8000/8000 [==============================] - 73s 9ms/step - loss: 2.6522 - acc: 0.3357 - val_loss: 2.4993 - val_acc: 0.3665
Epoch 5/15
8000/8000 [==============================] - 73s 9ms/step - loss: 2.4972 - acc: 0.3579 - val_loss: 2.3822 - val_acc: 0.3780
Epoch 6/15
8000/8000 [==============================] - 73s 9ms/step - loss: 2.3592 - acc: 0.3867 - val_loss: 2.3050 - val_acc: 0.3810
Epoch 7/15
8000/8000 [==============================] - 73s 9ms/step - loss: 2.2511 - acc: 0.4096 - val_loss: 2.2182 - val_acc: 0.3985
Epoch 

In [54]:
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 21s 10ms/step


[2.094347032546997, 0.43]